In [ ]:
using HumanoidLCMSim
using RigidBodySim
using RigidBodyDynamics
using AtlasRobot
using OrdinaryDiffEq
using DiffEqCallbacks
using RigidBodyTreeInspector
using RigidBodyDynamics.Contact

In [2]:
mechanism = AtlasRobot.mechanism(remove_fixed_tree_joints = false)
actuatorconfig = parse_actuators(mechanism, AtlasRobot.urdfpath())
remove_fixed_tree_joints!(mechanism);

In [3]:
# add environment
rootframe = root_frame(mechanism)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism, ground);

In [4]:
# Create visualizer
any_open_visualizer_windows() || (new_visualizer_window(); sleep(1));
vis = Visualizer()[:atlas]
setgeometry!(vis, mechanism, RigidBodyTreeInspector.parse_urdf(AtlasRobot.urdfpath(), mechanism; package_path = [AtlasRobot.packagepath()]))

In [5]:
feet = Dict(Sides.left => findbody(mechanism, "l_foot"), Sides.right => findbody(mechanism, "r_foot"))
hands = Dict(Sides.left => findbody(mechanism, "l_hand"), Sides.right => findbody(mechanism, "r_hand"))
robot_info = HumanoidRobotInfo(mechanism, feet, hands, actuatorconfig);

In [6]:
state = MechanismState(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [7]:
function initialize(state::MechanismState, atlas::Mechanism, vis::Visualizer)
    zero!(state)
    kneebend = 1.1
    hipbendextra = 0.1
    for sideprefix in ('l', 'r')
        set_configuration!(state, findjoint(atlas, "$(sideprefix)_leg_kny"), [kneebend])
        set_configuration!(state, findjoint(atlas, "$(sideprefix)_leg_hpy"), [-kneebend / 2 + hipbendextra])
        set_configuration!(state, findjoint(atlas, "$(sideprefix)_leg_aky"), [-kneebend / 2 - hipbendextra])
    end
    floatingjoint = first(out_joints(root_body(mechanism), mechanism))
    set_configuration!(state, floatingjoint, [1; 0; 0; 0; 0; 0; 0.85])
    settransform!(vis, state)
end

initialize (generic function with 1 method)

In [8]:
lcmcontroller = LCMController(robot_info)
controller = PeriodicController(1 / 300, lcmcontroller)
initialize(state, mechanism, vis)
final_time = 1.
problem = ODEProblem(state, (0., final_time), controller)
# sol = solve(problem, Tsit5(), abs_tol = 1e-10, dt = 0.05, callback = CallbackSet(vis, state));

In [9]:
# animate(vis, state, sol)

In [10]:
# function testfunc(controller, τ, t, state, n)
#     for i = 1 : n
#         controller(τ, t, state)
#     end
# end

# testfunc(controller, τ, t, state, 1)
# Profile.clear_malloc_data()
# @profile testfunc(controller, τ, t, state, 100000)

# using ProfileView
# ProfileView.view()